In [1]:
import copy

import re

from search import *

In [2]:
class Movix(Problem):
    N = 0
    Y = 0
    muros = []

    def __init__(self, N, Y, initial, goal=None, muros = []):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self.N = N
        self.Y = Y
        self._actions = []
        self.muros = muros

        for i in range(1, N):
            self._actions.append(('V', (0, i)))
            self._actions.append(('V', (0, -i)))

        for i in range(1, N):
            self._actions.append(('H', (i, 0)))
            self._actions.append(('H', (-i, 0)))

        for i in range(-1, 2):
            for j in range(-1, 2):
                if i != 0 or j != 0:
                    self._actions.append(('L', (i, j)))
                    if i == 0 or j == 0:
                        self._actions.append(('S', (2 * i, 2 * j)))

    def mostrarAcciones(self):
        print(self._actions)

    def actions(self, estado):
        tupla1 = estado[0]
        tupla2 = estado[1]
        tupla3 = estado[2]

        diccionario1 = dict(tupla1)
        for f in diccionario1:
            diccionario1[f] = list(diccionario1[f])

        diccionario2 = dict(tupla2)
        for f in diccionario2:
            diccionario2[f] = dict(diccionario2[f])

        diccionario3 = dict(tupla3)
        for f in diccionario3:
            diccionario3[f] = dict(diccionario3[f])

        s = (diccionario1, diccionario2, diccionario3)
        actions_list = []
        for a in self._actions:
            for pos in s[0][a[0]]:
                if self._is_valid(a, pos, s):
                #if self.esValido(a, pos, s):
                    actions_list.append("Mover ficha {} de ({},{}) a ({},{})".format(a[0], pos[0], pos[1], (pos[0] + a[1][0]), (pos[1] + a[1][1])))

        return actions_list

    #Nuevo is_valid con la condicion adicional de los muros
    def _is_valid(self, a, pos, estado):

        tupla1 = estado[0]
        tupla2 = estado[1]
        tupla3 = estado[2]

        diccionario1 = dict(tupla1)
        for f in diccionario1:
            diccionario1[f] = list(diccionario1[f])

        diccionario2 = dict(tupla2)
        for f in diccionario2:
            diccionario2[f] = dict(diccionario2[f])

        diccionario3 = dict(tupla3)
        for f in diccionario3:
            diccionario3[f] = dict(diccionario3[f])

        s = (diccionario1, diccionario2, diccionario3)

        fila, columna = (pos[0] + a[1][0], pos[1] + a[1][1])
        #Compruebo que la posicion este dentro de las dimensiones del tablero, que no hay ninguna ficha
        #en dicha posicion y que tampoco haya ningun muro
        if (fila < self.N and fila >= 0 and columna < self.N and columna >= 0 and 
            (fila not in s[1] or columna not in s[1][fila]) and self.muros.count((fila, columna)) == 0):
            match a[0]:
                case 'V':
                    return not(fila in s[1] and (any(pos[1] < y < columna for y in s[1][fila]) or any(columna < y < pos[1] for y in s[1][fila])))
                case 'H':
                    return not(columna in s[2] and (any(pos[0] < x < fila for x in s[2][columna]) or any(fila < x < pos[0] for x in s[2][columna])))
                case 'S':
                    #Como la ficha S no puede saltar muros compruebo adicionalmente que no haya un muro en la casilla adyacente a S
                    return not(not (pos[0] + a[1][0]/2 in s[1] and (pos[1] + a[1][1]/2) in s[1][pos[0] + a[1][0]/2])) and self.muros.count((pos[0] + a[1][0]/2, pos[1] + a[1][1]/2)) == 0
                case 'L':
                    return True
        else:
            return False
        
    #Antiguo is_valid
    def esValido(self, a, pos, s):
        fil, col = (pos[0] + a[1][0], pos[1] + a[1][1])
         # Verificamos si la ficha está dentro de los límites del tablero
        if fil >= self.N or fil < 0 or col >= self.N or col < 0:
            return False  # Si alguna ficha está fuera de los límites, el estado no es válido
        if fil in s[1] and col in s[1][fil]:
            return False
        if a[0] == 'V' and fil in s[1] and (any(pos[1] < y < col for y in s[1][fil]) or any(col < y < pos[1] for y in s[1][fil])):
            return False
        if a[0] == 'H' and col in s[2] and (any(pos[0] < x < fil for x in s[2][col]) or any(fil < x < pos[0] for x in s[2][col])):
            return False
        if a[0] == 'S' and not (pos[0] + a[1][0]/2 in s[1] and (pos[1] + a[1][1]/2) in s[1][pos[0] + a[1][0]/2]):
            return False
        return True       # Si todas las fichas están dentro de los límites, y cumplen las propiedades el estado es válido

    def result(self, estado, action):

        tupla1 = estado[0]
        tupla2 = estado[1]
        tupla3 = estado[2]

        diccionario1 = dict(tupla1)
        for f in diccionario1:
            diccionario1[f] = list(diccionario1[f])

        diccionario2 = dict(tupla2)
        for f in diccionario2:
            diccionario2[f] = dict(diccionario2[f])

        diccionario3 = dict(tupla3)
        for f in diccionario3:
            diccionario3[f] = dict(diccionario3[f])

        s = (diccionario1, diccionario2, diccionario3)

        patron = r"Mover ficha (\w) de \((\d+),(\d+)\) a \((\d+),(\d+)\)"
        # Buscar los valores
        resultado = re.findall(patron, action)

        if resultado:
            ficha, x1, y1, x2, y2 = resultado[0]
            x1, y1 = int(x1), int(y1)
            x2, y2 = int(x2), int(y2)

            new_s = copy.deepcopy(s)

            new_s[0][ficha].remove((x1, y1))
            #no deberia hacer falta comprobar con estos ifs 
            if y1 in new_s[2] and x1 in new_s[2][y1]:
                new_s[2][y1].pop(x1)
            if x1 in new_s[1] and y1 in new_s[1][x1]:
                new_s[1][x1].pop(y1)

            
            if x1 in new_s[1] and len(new_s[1][x1]) == 0:
                del new_s[1][x1]
            if y1 in new_s[2] and len(new_s[2][y1]) == 0:
                del new_s[2][y1]

            
            if x2 not in new_s[1]:
                new_s[1][x2] = {}

            if y2 not in new_s[2]:
                new_s[2][y2] = {}

            new_s[0][ficha].append((x2, y2))
            new_s[1][x2][y2] = ficha
            new_s[2][y2][x2] = ficha

            #nos aseguramos q todo este ordenado
            d1 = new_s[0]
            d2 = {k: new_s[1][k] for k in sorted(new_s[1].keys())}
            d3 = {k: new_s[2][k] for k in sorted(new_s[2].keys())}

            l1 = []
            for f, p in d1.items():
                l1.append((f, tuple(x for x in p)))

            l2 = []
            for x, d in d2.items():
                l2.append((x, tuple(a for a in d.items())))

            l3 = []
            for y, d in d3.items():
                l3.append((y, tuple(a for a in d.items())))
            
            #return new_s
            return (tuple(l1), tuple(l2), tuple(l3))
        else:
            return None

    def goal_test(self, estado):

        tupla1 = estado[0]
        tupla2 = estado[1]
        tupla3 = estado[2]

        diccionario1 = dict(tupla1)
        for f in diccionario1:
            diccionario1[f] = list(diccionario1[f])

        diccionario2 = dict(tupla2)
        for f in diccionario2:
            diccionario2[f] = dict(diccionario2[f])

        diccionario3 = dict(tupla3)
        for f in diccionario3:
            diccionario3[f] = dict(diccionario3[f])

        s = (diccionario1, diccionario2, diccionario3)

        #linea = (len(s[1]) == 1 and max(s[2].keys()) - min(s[2].keys()) == self.Y*4 - 1) or (len(s[2]) == 1 and max(s[1].keys()) - min(s[1].keys()) == self.Y*4 - 1)
        linea = len(s[1]) == 1 or len(s[2]) == 1

        diagonal = True
        #if len(s[1]) > 0 and len(s[1][min(s[1])]) > 0:
        clave_minima = min(s[1])
        diferencia = clave_minima - min(s[1][clave_minima])
        for i in s[1]:
            if(len(s[1][i]) > 1):
                diagonal = False
                break
            elif(i - min(s[1][i]) != diferencia):
                diagonal = False
                break

        return linea or diagonal

        

In [3]:
N = int(input("Introduzca el numero de casillas que tendra el tablero: "))
Y = int(input("Introduzca el numero de fichas que habra en el tablero: "))
M = int(input("Cuantos muros desea que haya en el tablero: "))

while(Y > N):
    print("El numero de fichas no puede ser mayor a las dimensiones del tablero")
    N = int(input("Introduzca el numero de casillas que tendra el tablero: "))
    Y = int(input("Introduzca el numero de fichas que habra en el tablero"))

#Lista de posiciones para asegurar que no se repita ninguna posicion
posiciones = []

#Lista que almacena las posiciones de los muros
muros = []

#Diccionario que relaciona el tipo de ficha con la lista de las distintas posiciones donde estan
fichasPos = {'S' : [], 'H' : [], 'V' : [], 'L' : []}

for ficha in fichasPos:
    for i in range (0, Y):
        pos = (random.randrange(0, N), random.randrange(0, N))
        while(posiciones.count(pos) > 0):
            pos = (random.randrange(0, N), random.randrange(0, N))

        fichasPos[ficha].append(pos)
        posiciones.append(pos)

for m in range(0, M):
    muro = (random.randrange(0, N), random.randrange(0, N))
    while(posiciones.count(muro) > 0 or muros.count(muro) > 0):
        muro = (random.randrange(0, N), random.randrange(0, N))

    muros.append(muro)

#Diccionario que guarda todas las fichas que estan en una fila con la columna en la que esta y el tipo de ficha que es
xToFicha = {}
#Diccionario que guarda todas las fichas que estan en una columna con la fila en la que esta y el tipo de ficha que es
yToFicha = {}

for ficha, positions in fichasPos.items():
    for pos in positions:
        if pos[0] not in xToFicha:  xToFicha[pos[0]] = {pos[1] : ficha}
        else: xToFicha[pos[0]] = xToFicha[pos[0]] | {pos[1] : ficha}

        if pos[1] not in yToFicha:  yToFicha[pos[1]] = {pos[0] : ficha}
        else: yToFicha[pos[1]] = yToFicha[pos[1]] | {pos[0] : ficha}

#Inicializo la tupla con los tres diccionarios para almacenar el estado
estado = (fichasPos, xToFicha, yToFicha)
estado

({'S': [(2, 1)], 'H': [(3, 1)], 'V': [(1, 4)], 'L': [(3, 3)]},
 {2: {1: 'S'}, 3: {1: 'H', 3: 'L'}, 1: {4: 'V'}},
 {1: {2: 'S', 3: 'H'}, 4: {1: 'V'}, 3: {3: 'L'}})

In [4]:
#Traducir los diccionarios a tuplas
lista1 = []
for f, p in fichasPos.items():
    lista1.append((f, tuple(x for x in p)))

lista2 = []
for x, d in xToFicha.items():
    lista2.append((x, tuple(a for a in d.items())))

lista3 = []
for y, d in yToFicha.items():
    lista3.append((y, tuple(a for a in d.items())))

rayas = "---------------------------------"
#Primer diccionario pasado a tupla
tupla1 = tuple(lista1)
print(tupla1)
print(rayas)
#Segundo diccionario pasado a tupla
tupla2 = tuple(lista2)
print(tupla2)
print(rayas)
#Tercer diccionario pasado a tupla
tupla3 = tuple(lista3)
print(tupla3)
print(rayas)

(('S', ((2, 1),)), ('H', ((3, 1),)), ('V', ((1, 4),)), ('L', ((3, 3),)))
---------------------------------
((2, ((1, 'S'),)), (3, ((1, 'H'), (3, 'L'))), (1, ((4, 'V'),)))
---------------------------------
((1, ((2, 'S'), (3, 'H'))), (4, ((1, 'V'),)), (3, ((3, 'L'),)))
---------------------------------


In [5]:
#Volver a convertir las tuplas a los diccionarios originales
diccionarioOtraVez1 = dict(list(tupla1))
print(diccionarioOtraVez1)
print("El problema es que el diccionario se queda como valor una tupla de tuplas y no una lista de tuplas")
#Asi que para cada tupla de tuplas del diccionario tengo que convertirla a una lista
for f in diccionarioOtraVez1:
    diccionarioOtraVez1[f] = list(diccionarioOtraVez1[f])
print(diccionarioOtraVez1)
print("Ya ha vuelto a como estaba antes")
print(rayas)

diccionarioOtraVez2 = dict(list(tupla2))
print(diccionarioOtraVez2)
print("El problema es el mismo que el del diccionario anterior, pero ahora con diccionarios en vez de listas")
for f in diccionarioOtraVez2:
    diccionarioOtraVez2[f] = dict(diccionarioOtraVez2[f])

print(diccionarioOtraVez2)
print("Problema arreglado")
print(rayas)

diccionarioOtraVez3 = dict(list(tupla3))
for f in diccionarioOtraVez3:
    diccionarioOtraVez3[f] = dict(diccionarioOtraVez3[f])
print(diccionarioOtraVez3)

{'S': ((2, 1),), 'H': ((3, 1),), 'V': ((1, 4),), 'L': ((3, 3),)}
El problema es que el diccionario se queda como valor una tupla de tuplas y no una lista de tuplas
{'S': [(2, 1)], 'H': [(3, 1)], 'V': [(1, 4)], 'L': [(3, 3)]}
Ya ha vuelto a como estaba antes
---------------------------------
{2: ((1, 'S'),), 3: ((1, 'H'), (3, 'L')), 1: ((4, 'V'),)}
El problema es el mismo que el del diccionario anterior, pero ahora con diccionarios en vez de listas
{2: {1: 'S'}, 3: {1: 'H', 3: 'L'}, 1: {4: 'V'}}
Problema arreglado
---------------------------------
{1: {2: 'S', 3: 'H'}, 4: {1: 'V'}, 3: {3: 'L'}}


In [6]:
def h_1(node):
    estado = node.state
    tupla1 = estado[0]
    tupla2 = estado[1]
    tupla3 = estado[2]

    diccionario1 = dict(tupla1)
    for f in diccionario1:
        diccionario1[f] = list(diccionario1[f])

    diccionario2 = dict(tupla2)
    for f in diccionario2:
        diccionario2[f] = dict(diccionario2[f])

    diccionario3 = dict(tupla3)
    for f in diccionario3:
        diccionario3[f] = dict(diccionario3[f])

    s = (diccionario1, diccionario2, diccionario3)

    # Calculamos el número de fichas restantes para hacer una fila en X e Y
    left_x = Y * 4 - len(s[1]) if s[1] else Y * 4
    left_y = Y * 4 - len(s[2]) if s[2] else Y * 4
    left_diag = 0

    # Calculamos el número de fichas restantes para hacer una diagonal
    if s[1]:  # Verificar que s[1] no esté vacío
        clave_minima = min(s[1].keys())
        diferencia = clave_minima - min(s[1][clave_minima]) if s[1][clave_minima] else 0
        for i in s[1]:
            if s[1][i] and (i - min(s[1][i]) != diferencia):  # Verifica que s[1][i] no esté vacío
                left_diag += 1

    return min(left_x, left_y, left_diag)
        

In [7]:
d1 =  {'S': [(1, 1)], 'H': [(3, 1)], 'L': [(2, 2)], 'V': [(0, 3)]}
d2 = {0: {3: 'V'}, 1: {1: 'S'}, 2: {2: 'L'}, 3: {1: 'H'}}
d3 = {1: {1: 'S', 3: 'H'}, 2: {2: 'L'}, 3: {0: 'V'}}

l1 = []
for f, p in d1.items():
    l1.append((f, tuple(x for x in p)))

l2 = []
for x, d in d2.items():
    l2.append((x, tuple(a for a in d.items())))

l3 = []
for y, d in d3.items():
    l3.append((y, tuple(a for a in d.items())))

s = (tuple(l1), tuple(l2), tuple(l3))
p = Movix(N, Y, s, muros)

p.actions(p.initial)

['Mover ficha V de (0,3) a (0,4)',
 'Mover ficha V de (0,3) a (0,2)',
 'Mover ficha V de (0,3) a (0,1)',
 'Mover ficha V de (0,3) a (0,0)',
 'Mover ficha H de (3,1) a (4,1)',
 'Mover ficha H de (3,1) a (2,1)',
 'Mover ficha L de (2,2) a (1,2)',
 'Mover ficha L de (2,2) a (1,3)',
 'Mover ficha L de (2,2) a (2,1)',
 'Mover ficha L de (2,2) a (2,3)',
 'Mover ficha L de (2,2) a (3,2)',
 'Mover ficha L de (2,2) a (3,3)']

In [8]:
%%time
breadth_first_tree_search(p).solution()

CPU times: total: 15.6 ms
Wall time: 61.5 ms


['Mover ficha V de (0,3) a (0,1)', 'Mover ficha L de (2,2) a (2,1)']

In [9]:
%%time
astar_search(p, h_1).solution()

CPU times: total: 15.6 ms
Wall time: 25 ms


['Mover ficha V de (0,3) a (0,1)', 'Mover ficha L de (2,2) a (2,1)']

In [10]:
s = (
    {'S': [(0, 2)], 'H': [(0, 3)], 'L': [(4, 3)], 'V': [(2, 3)]},
    {0: {3: 'H', 2: 'S'}, 2: {3: 'V'}, 4: {3: 'L'}},
    {2: {0: 'S'},3: {0: 'H', 2: 'V', 4: 'L'}}
    )
p = Movix(N, Y, s, muros)

In [11]:
%%time
breadth_first_tree_search(p).solution()

KeyboardInterrupt: 

In [45]:
s = (
    {'S': [(0, 4)], 'H': [(0, 3)], 'L': [(4, 3)], 'V': [(2, 3)]},
    {0: {3: 'H', 4: 'S'}, 2: {3: 'V'}, 4: {3: 'L'}},
    {3: {0: 'H', 2: 'V', 4: 'L'},4: {0: 'S'}}
    )
p = Movix(N, Y, s, muros)

In [46]:
breadth_first_tree_search(p).solution()

TypeError: tuple indices must be integers or slices, not tuple

In [ ]:
s = (
    {'S': [(1, 7), (3, 7)], 'H': [(1, 5), (8, 2)], 'L': [(4, 1), (3, 9)], 'V': [(4, 6), (6, 7)]},
    {1: {5: 'H', 7: 'S'}, 3: {7: 'S', 9: 'L'}, 4: {1: 'L', 6: 'V'}, 6: {7: 'V'}, 8: {2: 'H'}},
    {1: {6: 'L'}, 2: {8: 'H'}, 5: {1: 'H'}, 6: {4: 'V'}, 7: {1: 'S', 3: 'S', 6: 'V'}, 9: {3: 'L'}}
)

p = Movix(10, 2, s)

In [ ]:
breadth_first_tree_search(p).solution()